In [ ]:
import os
import torch
import scipy.stats as stats
import numpy as np

# CODE FILES HERE
from model_params import get_model_data_cvae
from models.cvae.cvae import Cvae
from solver import Solver
from directories import Directories
from dataloader import DataLoader
from plots import plot_losses, plot_gaussian_distributions, plot_rl_kl, plot_latent_space, \
plot_with_fixed_z, plot_latent_manifold, plot_faces_grid, plot_faces_samples_grid
from auxiliary import get_latent_spaces

# SETTINGS HERE
os.environ['CUDA_LAUNCH_BLOCKING'] = "1" # to see the CUDA stack
%matplotlib inline
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
# supress cluttering warnings in solutions
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [ ]:
# Choose only MNIST
dataset_arg = "MNIST"
data = get_model_data_cvae(dataset_arg)

In [ ]:
directories = Directories("cvae", dataset_arg.lower(), data["z_dim"], make_dirs=False)
data_loader = DataLoader(directories, data["batch_size"], dataset_arg.lower())
model = Cvae(data_loader.input_dim, data["hidden_dim"], data["z_dim"], data["beta"], data_loader.n_classes, data["batch_norm"])
solver = Solver(model, data_loader, data["optimizer"], data["epochs"], data["optim_config"], step_config=data["step_config"], lr_scheduler=data["lr_scheduler"], cvae_mode=True, save_model_state=False)
solver.main()

In [ ]:
# Insert name of model here if want to load a model, e.g. solver.save_model_dir + "/VAE_MNIST_train_loss=151.39_z=2.pt"
# solver = torch.load(solver.save_model_dir + "/CVAE_MNIST_train_loss=135.66_z=2" + ".pt")
# solver.model.eval()

In [ ]:
# Plotting train and test losses for all epochs
plot_losses(solver, solver.train_loss_history["train_loss_acc"], solver.test_loss_history)

In [ ]:
# Plotting the gaussian of z space and some metrics about the space
plot_gaussian_distributions(solver, len(solver.train_loss_history["train_loss_acc"]))

In [ ]:
# Monitoring the reconstruction loss (likelihood lower bound) and KL divergence
DEBUG = 0
if DEBUG:
    for epoch, train_loss, test_loss, rl, kl in zip(solver.train_loss_history["epochs"], \
                             solver.train_loss_history["train_loss_acc"], \
                             solver.test_loss_history, \
                             solver.train_loss_history["recon_loss_acc"], \
                             solver.train_loss_history["kl_diverg_acc"]):
        print("epoch: {}, train_loss: {:.2f}, test_loss: {:.2f}, recon. loss: {:.2f}, KL div.: {:.2f}".format(
            epoch, train_loss, test_loss, rl, kl))
plot_rl_kl(solver, solver.train_loss_history["recon_loss_acc"], solver.train_loss_history["kl_diverg_acc"])

In [ ]:
# visualize q(z) (latent space z)
if solver.model.z_dim == 2:
    z_space, _, labels = get_latent_spaces(solver)
    if solver.data_loader.with_labels:
        plot_latent_space(solver, z_space, var="z", title="classes", labels=labels.tolist())
    else:
        plot_latent_space(solver, z_space, var="z")
else:
    print("Plot is not possible as dimension of z is not 2")

In [ ]:
# Visualizations of learned data manifold for generative models with two-dimensional latent space
if solver.model.z_dim == 2:
    n = 20
    grid_x = grid_y = np.linspace(-3, 3, n)
    plot_latent_manifold(solver, "bone", grid_x, grid_y, n)
else:
    print("Plot is not possible as dimension of z is not 2")

In [ ]:
# Plotting a row of squares with each 11 squares, where first is the base for the style
plot_with_fixed_z(solver, "gray")